In [1]:
import os
import sys

sys.path.append("../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from src.utils.helper import Config, color_print
from src.utils.load import load_model, load_data, save_checkpoint, load_checkpoint
from src.models.evaluate import evaluate_model, get_sparsity, get_similarity
from src.utils.sampling import SamplingDataset


In [3]:
name = "bert-4-128-yahoo"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 128
ratio = 0.5
seed = 44

In [4]:
config = Config(name, device)

In [5]:
model = load_model(config=config)

Loading the model.
{'architectures': 'bert',
 'dataset_name': 'YahooAnswersTopics',
 'model_name': 'models/bert-4-128-yahoo',
 'num_labels': 10,
 'tokenizer_name': 'fabriceyhc/bert-base-uncased-yahoo_answers_topics'}
The model models/bert-4-128-yahoo is loaded.


In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    config,
    batch_size=batch_size,
    num_workers=num_workers,
    do_cache=True,
)

Loading cached dataset YahooAnswersTopics.
train.pkl is loaded from cache.
valid.pkl is loaded from cache.
test.pkl is loaded from cache.
The dataset YahooAnswersTopics is loaded
{'config_name': 'yahoo_answers_topics',
 'features': {'first_column': 'question_title', 'second_column': 'topic'},
 'path': 'yahoo_answers_topics'}


In [7]:
import copy
import torch
import numpy as np
from typing import *
import torch.nn as nn
from functools import partial
from transformers.pytorch_utils import (
    find_pruneable_heads_and_indices,
    prune_linear_layer,
)


def calculate_head_importance(
    model,
    config,
    data,
    normalize_scores_by_layer=True,
):
    device = config.device
    from functools import partial

    gradients = {}
    context_layers = {}

    def save_grad(gradients, layer_idx, grad):
        gradients[f"context_layer_{layer_idx}"] = grad

    def forward_hook(module, input, output, gradients, context_layers, layer_idx):
        context_layers[f"context_layer_{layer_idx}"] = output[0]
        output[0].register_hook(partial(save_grad, gradients, layer_idx))

    def reshape(tensors, shape, num_heads):
        batch_size = shape[0]
        seq_len = shape[1]
        head_dim = shape[2] // num_heads
        tensors = tensors.reshape(batch_size, seq_len, num_heads, head_dim)
        tensors = tensors.permute(0, 2, 1, 3)
        return tensors

    forward_handles = []

    for layer_idx in range(model.bert.config.num_hidden_layers):
        self_att = model.bert.encoder.layer[layer_idx].attention.self
        handle = self_att.register_forward_hook(
            partial(
                forward_hook,
                gradients=gradients,
                context_layers=context_layers,
                layer_idx=layer_idx,
            )
        )
        forward_handles.append(handle)

    """Calculate head importance scores"""
    # Disable dropout
    model.eval()
    # Device
    device = device or next(model.parameters()).device

    # Prepare data loader
    # Head importance tensor
    n_layers = model.bert.config.num_hidden_layers
    n_heads = model.bert.config.num_attention_heads
    num_batches = 0
    head_dim = model.bert.config.hidden_size // n_heads
    head_importance = torch.zeros(n_layers, n_heads).to(device)
    tot_tokens = 0
    first_batch = next(iter(data))
    is_embeds = "embeddings" in first_batch
    for step, batch in enumerate(data):
        num_batches += 1
        if is_embeds:
            embeddings = batch["embeddings"].to(device)
        else:
            input_ids = batch["input_ids"].to(device)
        input_mask = batch["attention_mask"].to(device)
        label_ids = batch["labels"].to(device)
        # Compute gradients
        if is_embeds:
            loss = model(
                inputs_embeds=embeddings, attention_mask=input_mask, labels=label_ids
            ).loss
        else:
            loss = model(input_ids, attention_mask=input_mask, labels=label_ids).loss
        loss.backward()

        for layer_idx in range(model.bert.config.num_hidden_layers):
            ctx = context_layers[f"context_layer_{layer_idx}"]
            grad_ctx = gradients[f"context_layer_{layer_idx}"]
            shape = ctx.shape
            ctx = reshape(ctx, shape, n_heads)
            grad_ctx = reshape(grad_ctx, shape, n_heads)

            # Take the dot
            dot = torch.einsum("bhli,bhli->bhl", [grad_ctx, ctx])
            head_importance[layer_idx] += dot.abs().sum(-1).sum(0).detach()
            del ctx, grad_ctx, dot

        tot_tokens += input_mask.float().detach().sum().data

    head_importance[:-1] /= tot_tokens
    head_importance /= num_batches
    for handle in forward_handles:
        handle.remove()
    return head_importance

def calculate_head_loss(model, config, data):
    device = config.device
    model.eval()
    n_layers = model.bert.config.num_hidden_layers
    n_heads = model.bert.config.num_attention_heads
    is_embeds = "embeddings" in next(iter(data))

    head_losses = torch.zeros(n_layers, n_heads).to(device)
    num_samples = 0
    
    for step, batch in enumerate(data):
        if is_embeds:
            embeddings = batch["embeddings"].to(device)
        else:
            input_ids = batch["input_ids"].to(device)
        input_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        batch_size = input_mask.size(0)
        
        for batch_item in range(batch_size):
            num_samples += 1
            for layer in range(n_layers):
                for head in range(n_heads):
                    head_mask = torch.ones(n_layers, n_heads).to(device)
                    head_mask[layer, head] = 0

                    with torch.no_grad():
                        if is_embeds:
                            outputs = model(inputs_embeds=embeddings, attention_mask=input_mask, head_mask=head_mask, output_attentions=True)
                        else:                
                            outputs = model(input_ids=input_ids, attention_mask=input_mask, head_mask=head_mask, output_attentions=True)
                            
                    logits = outputs.logits
                    criterion = torch.nn.CrossEntropyLoss()
                    loss = criterion(logits[batch_item].unsqueeze(0), labels[batch_item].unsqueeze(0))
                    head_losses[layer, head] += loss.item()

    avg_loss = head_losses / num_samples

    return avg_loss

def calculate_layer_loss(model, config, data):
    device = config.device
    model.eval()
    n_layers = model.bert.config.num_hidden_layers
    n_heads = model.bert.config.num_attention_heads
    is_embeds = "embeddings" in next(iter(data))

    layer_losses = torch.zeros(n_layers).to(device)
    num_samples = 0

    for step, batch in enumerate(data):
        if is_embeds:
            embeddings = batch["embeddings"].to(device)
        else:
            input_ids = batch["input_ids"].to(device)
        input_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        batch_size = input_mask.size(0)
        
        for batch_item in range(batch_size):
            num_samples += 1
            for layer in range(n_layers):
            
                head_mask = torch.ones(n_layers).to(device)
                head_mask[layer] = 0

                with torch.no_grad():
                    if is_embeds:
                        outputs = model(inputs_embeds=embeddings, attention_mask=input_mask, head_mask=head_mask, output_attentions=True)
                    else:                
                        outputs = model(input_ids=input_ids, attention_mask=input_mask, head_mask=head_mask, output_attentions=True)
                        
                logits = outputs.logits
                criterion = torch.nn.CrossEntropyLoss()
                loss = criterion(logits[batch_item].unsqueeze(0), labels[batch_item].unsqueeze(0))
                layer_losses[layer] += loss.item()

    avg_loss = layer_losses / num_samples

    return avg_loss


def normalize(tensors):
    exponent = 2
    norm_by_layer = torch.pow(
        torch.pow(tensors, exponent).sum(-1), 1 / exponent
    )
    tensors /= norm_by_layer.unsqueeze(-1) + 1e-20
    return tensors

def head_importance_prunning(
    model, config, dominant_concern, sparsity_ratio, method="unstructed", scheduler=None
):
    num_attention_heads = model.config.num_attention_heads
    num_hidden_layers = model.config.num_hidden_layers
    model = model.to(config.device)
    total_heads_to_prune = int(num_attention_heads * num_hidden_layers * sparsity_ratio)

    total_heads_to_prune = max(total_heads_to_prune, num_hidden_layers)
    print(f"Total heads to prune: {total_heads_to_prune}")
    pruned_heads = set()

    if scheduler is not None:
        steps = scheduler.get_steps()
    else:
        steps = [0.25, 0.25, 0.25, 0.25]
        # steps = [1.0]

    for step_ratio in steps:
        heads_to_prune = int(total_heads_to_prune * step_ratio)
        
        head_importance_list = calculate_head_importance(
            model, config, dominant_concern
        )
        head_loss = calculate_head_loss(
            model, config, dominant_concern
        )
        layer_loss = calculate_layer_loss(
            model, config, dominant_concern
        )
        head_importance_list = head_importance_list.cpu()
        head_loss = head_loss.cpu()
        layer_loss = layer_loss.cpu()
        # head_importance_list = normalize(head_importance_list)
        # head_loss = normalize(head_loss)
        # layer_loss = normalize(layer_loss)
        head_importance_list = torch.log1p(head_importance_list)
        print(f"head_importance_list\n {head_importance_list}")
        print(f"head_loss\n {head_loss}")
        print(f"layer_loss\n {layer_loss}")
        alpha = head_loss * layer_loss.unsqueeze(1)
        print(alpha)

        head_score = head_importance_list * alpha
        print(f"head_score\n {head_score}")
        if method == "unstructed":
            sorted_indices = torch.argsort(head_score.view(-1))
            prune_list = []
            for idx in sorted_indices:
                layer_index = int(idx // num_attention_heads)
                head_index = int(idx % num_attention_heads)

                if (layer_index, head_index) not in pruned_heads:
                    prune_list.append((layer_index, head_index))
                
                if len(prune_list) >= heads_to_prune:
                    break
            
        elif method == "structed":
            heads_per_layer = heads_to_prune // num_hidden_layers
            prune_list = []
            for layer_idx in range(num_hidden_layers):
                sorted_heads = torch.argsort(head_importance_list[layer_idx])
                prune_list.extend(
                    [
                        (layer_idx, head.item())
                        for head in sorted_heads[:heads_per_layer]
                    ]
                )
        for layer_index, head_index in prune_list:
            if (layer_index, head_index) not in pruned_heads:
                prune_heads(
                    model.bert.encoder.layer[layer_index].attention,
                    [head_index],
                    method=method,
                )
                pruned_heads.add((layer_index, head_index))
        print(sorted(pruned_heads))


def prune_heads(layer, heads, method):
    if len(heads) == 0:
        return
    heads, index = find_pruneable_heads_and_indices(
        heads,
        layer.self.num_attention_heads,
        layer.self.attention_head_size,
        layer.pruned_heads,
    )

    # if method == "unstructed":
    layer.self.query = zero_out_head_weights(
        layer.self.query, heads, layer.self.attention_head_size
    )
    layer.self.key = zero_out_head_weights(
        layer.self.key, heads, layer.self.attention_head_size
    )
    layer.self.value = zero_out_head_weights(
        layer.self.value, heads, layer.self.attention_head_size
    )
    layer.output.dense = zero_out_head_weights(
        layer.output.dense, heads, layer.self.attention_head_size, dim=1
    )
    # elif method == "structed":
    #     layer.self.query = prune_linear_layer(layer.self.query, index)
    #     layer.self.key = prune_linear_layer(layer.self.key, index)
    #     layer.self.value = prune_linear_layer(layer.self.value, index)
    #     layer.output.dense = prune_linear_layer(layer.output.dense, index)

    #     layer.self.num_attention_heads = layer.self.num_attention_heads - len(heads)
    #     layer.self.all_head_size = layer.self.attention_head_size *  layer.self.num_attention_heads
    #     layer.pruned_heads = layer.pruned_heads.union(heads)


def zero_out_head_weights(
    layer: nn.Linear, heads: Set[int], head_size: int, dim: int = 0
) -> nn.Linear:
    """
    Zero out the weights of the specified heads in the linear layer.

    Args:
        layer (`torch.nn.Linear`): The layer to modify.
        heads (`Set[int]`): The indices of heads to zero out.
        head_size (`int`): The size of each head.
        dim (`int`, *optional*, defaults to 0): The dimension on which to zero out the weights.

    Returns:
        `torch.nn.Linear`: The modified layer with weights of specified heads zeroed out.
    """
    for head in heads:
        start_index = head * head_size
        end_index = (head + 1) * head_size
        if dim == 0:
            layer.weight.data[start_index:end_index] = 0
        elif dim == 1:
            layer.weight.data[:, start_index:end_index] = 0

    return layer


In [8]:
from src.models.evaluate import (
    evaluate_model,
    get_sparsity,
    get_similarity,
    get_perplexity,
)

In [9]:
result_list = []

for concern in range(config.num_labels):
  config.init_seed()
  positive_samples = SamplingDataset(
      train_dataloader,
      config,
      concern,
      num_samples,
      True,
      4,
      resample=False,
  )
  negative_samples = SamplingDataset(
      train_dataloader,
      config,
      concern,
      num_samples,
      False,
      4,
      resample=False,
  )
  module = copy.deepcopy(model)

  head_importance_prunning(module, config, positive_samples, ratio)

#   print(f"Evaluate the pruned model {concern}")
  result = evaluate_model(module, config, test_dataloader)
#   result_list.append(result)
#   get_similarity(model, module, valid_dataloader, concern, num_samples, config)
#   print("original model's perplexity")
#   get_perplexity(model, valid_dataloader, config)
#   print("pruned model's perplexity")
#   get_perplexity(module, valid_dataloader, config)
  

Total heads to prune: 8


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


head_importance_list
 tensor([[5.6371e-05, 6.4266e-05, 5.7026e-05, 5.8088e-05],
        [4.8789e-05, 8.0193e-05, 6.4501e-05, 6.9048e-05],
        [5.0231e-05, 7.1948e-05, 5.1353e-05, 5.3254e-05],
        [5.9122e-02, 4.9883e-02, 6.9143e-02, 5.9519e-02]])
head_loss
 tensor([[1.4575, 1.4098, 1.4667, 1.5068],
        [1.4333, 1.4087, 1.4519, 1.4669],
        [1.4812, 1.4692, 1.4888, 1.4394],
        [1.5510, 1.4956, 1.4531, 1.4402]])
layer_loss
 tensor([1.4241, 1.3157, 1.3873, 1.3975])
tensor([[2.0757, 2.0078, 2.0887, 2.1459],
        [1.8858, 1.8534, 1.9101, 1.9299],
        [2.0548, 2.0382, 2.0654, 1.9969],
        [2.1675, 2.0901, 2.0307, 2.0127]])
head_score
 tensor([[1.1701e-04, 1.2903e-04, 1.1911e-04, 1.2465e-04],
        [9.2005e-05, 1.4863e-04, 1.2321e-04, 1.3325e-04],
        [1.0321e-04, 1.4664e-04, 1.0607e-04, 1.0634e-04],
        [1.2815e-01, 1.0426e-01, 1.4041e-01, 1.1980e-01]])
[(1, 0), (2, 0)]
head_importance_list
 tensor([[5.5054e-05, 6.4347e-05, 5.6189e-05, 5.9340e-05],
 

Evaluating the model:   0%|          | 0/1875 [00:00<?, ?it/s]

Loss: 1.4236
Precision: 0.6311, Recall: 0.5462, F1-Score: 0.5581
              precision    recall  f1-score   support

           0     0.4136    0.6120    0.4936      2992
           1     0.6786    0.4241    0.5220      2992
           2     0.6361    0.6361    0.6361      3012
           3     0.2936    0.6594    0.4063      2998
           4     0.7448    0.7302    0.7374      2973
           5     0.8999    0.4682    0.6160      3054
           6     0.6330    0.4049    0.4939      3003
           7     0.4639    0.6122    0.5278      3012
           8     0.7050    0.4624    0.5585      2982
           9     0.8427    0.4527    0.5890      2982

    accuracy                         0.5461     30000
   macro avg     0.6311    0.5462    0.5581     30000
weighted avg     0.6313    0.5461    0.5580     30000

Total heads to prune: 8
head_importance_list
 tensor([[4.9999e-05, 7.6041e-05, 4.8225e-05, 6.2064e-05],
        [3.4291e-05, 6.8904e-05, 4.9083e-05, 5.1879e-05],
        [4.253

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7314f88f8900>
Traceback (most recent call last):
  File "/home/jieungkim/.cache/pypoetry/virtualenvs/decomposetransformer-UESb9BbT-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/home/jieungkim/.cache/pypoetry/virtualenvs/decomposetransformer-UESb9BbT-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/home/jieungkim/anaconda3/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7314f88f8900>
Traceback (most recent call last):
  File "/home/jieungkim/.cache/pypoetry/virtualenvs/decompo

Evaluating the model:   0%|          | 0/1875 [00:00<?, ?it/s]

Loss: 1.4532
Precision: 0.6171, Recall: 0.5445, F1-Score: 0.5556
              precision    recall  f1-score   support

           0     0.3958    0.6243    0.4845      2992
           1     0.5960    0.4733    0.5276      2992
           2     0.6641    0.6066    0.6340      3012
           3     0.3000    0.6391    0.4083      2998
           4     0.8094    0.5658    0.6660      2973
           5     0.8992    0.3854    0.5395      3054
           6     0.6285    0.3989    0.4881      3003
           7     0.5098    0.5156    0.5127      3012
           8     0.6368    0.6402    0.6385      2982
           9     0.7309    0.5956    0.6563      2982

    accuracy                         0.5441     30000
   macro avg     0.6171    0.5445    0.5556     30000
weighted avg     0.6174    0.5441    0.5554     30000

Total heads to prune: 8
head_importance_list
 tensor([[5.7599e-05, 4.4358e-05, 4.0555e-05, 4.2634e-05],
        [4.4702e-05, 4.6824e-05, 4.9800e-05, 4.5123e-05],
        [3.168

Evaluating the model:   0%|          | 0/1875 [00:00<?, ?it/s]

Loss: 1.3779
Precision: 0.6321, Recall: 0.5727, F1-Score: 0.5846
              precision    recall  f1-score   support

           0     0.4045    0.6126    0.4873      2992
           1     0.6143    0.4499    0.5194      2992
           2     0.7165    0.5378    0.6145      3012
           3     0.3102    0.6481    0.4196      2998
           4     0.8340    0.5291    0.6475      2973
           5     0.8261    0.7449    0.7834      3054
           6     0.6487    0.3966    0.4923      3003
           7     0.5771    0.5631    0.5700      3012
           8     0.6858    0.5879    0.6331      2982
           9     0.7036    0.6566    0.6793      2982

    accuracy                         0.5729     30000
   macro avg     0.6321    0.5727    0.5846     30000
weighted avg     0.6323    0.5729    0.5849     30000

